# Phần 2: Tiền xử lí dữ liệu các tỉnh thành Việt Nam

### Bước 1: Gộp toàn bộ tập DL trong 7 ngày

In [1]:
import os
import pandas as pd

In [2]:
folder_path = './raw_data/'

all_data = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        data = pd.read_csv(file_path)
        all_data.append(data)

df = pd.concat(all_data)

df.head()

,ThoiGian,TinhThanh,KhuVuc,NhietDo (°C),LuongMua (mm),DoAm (%),DoAmTrongNha (%),KhiAp (mb),MatDoMay (%),TamNhin (km),...,PM2_Matdo(µg/m³),PM10_TrangThai,PM10_ChiSo,PM10_Matdo(µg/m³),SO2_TrangThai,SO2_ChiSo,SO2_Matdo(µg/m³),NO2_TrangThai,NO2_ChiSo,NO2_Matdo(µg/m³)
0,2024-06-07 17:43:57.759865+07:00,An Giang,Châu Đốc,34°C,2.9,55%,55% (Extremely Humid),↔ 1009 mb,86%,24 km,...,8 µg/m³,Fair,21,16 µg/m³,Excellent,12,36 µg/m³,Excellent,7,4 µg/m³
1,2024-06-07 17:44:05.521632+07:00,An Giang,Chợ Mới,33°C,7.9,63%,63% (Extremely Humid),↔ 1009 mb,77%,24 km,...,8 µg/m³,Fair,21,16 µg/m³,Excellent,13,39 µg/m³,Excellent,6,3 µg/m³
2,2024-06-07 17:44:08.978829+07:00,An Giang,Long Xuyên,33°C,7.9,63%,63% (Extremely Humid),↔ 1009 mb,77%,24 km,...,8 µg/m³,Fair,21,16 µg/m³,Excellent,13,39 µg/m³,Excellent,6,3 µg/m³
3,2024-06-07 17:44:13.347844+07:00,An Giang,Phú Châu,34°C,9.6,54%,54% (Extremely Humid),↔ 1009 mb,90%,24 km,...,9 µg/m³,Fair,25,20 µg/m³,Excellent,13,40 µg/m³,Excellent,7,4 µg/m³
4,2024-06-07 17:44:16.532666+07:00,An Giang,Tịnh Biên,32°C,6.1,63%,63% (Extremely Humid),↔ 1009 mb,76%,24 km,...,8 µg/m³,Fair,22,17 µg/m³,Excellent,13,38 µg/m³,Excellent,6,3 µg/m³


#### 1. Kiểm tra khuyết thiếu

In [3]:
df.isna().sum()

ThoiGian             0
TinhThanh            0
KhuVuc               0
NhietDo (°C)         0
LuongMua (mm)        0
DoAm (%)             0
DoAmTrongNha (%)     0
KhiAp (mb)           0
MatDoMay (%)         0
TamNhin (km)         0
AQI                  0
TrangThaiKhongKhi    0
PM2_TrangThai        0
PM2_ChiSo            0
PM2_Matdo(µg/m³)     0
PM10_TrangThai       0
PM10_ChiSo           0
PM10_Matdo(µg/m³)    0
SO2_TrangThai        0
SO2_ChiSo            0
SO2_Matdo(µg/m³)     0
NO2_TrangThai        0
NO2_ChiSo            0
NO2_Matdo(µg/m³)     0
dtype: int64

#### 2. Kiểm tra dữ liệu trùng lặp

In [4]:
df.duplicated().sum()

0

### Bước 2: Làm sạch dữ liệu

#### 2.1 Loại bỏ các đơn vị đi kèm với giá trị trong toàn bộ các cột

In [5]:
df['ThoiGian'] = pd.to_datetime(df['ThoiGian']).dt.date
df['NhietDo (°C)'] = df['NhietDo (°C)'].str.replace('°C', '').astype(float)
df['DoAm (%)'] = df['DoAm (%)'].str.replace('%', '').astype(int)
df['DoAmTrongNha (%)'] = df['DoAmTrongNha (%)'].str[:2].astype(int) #Do ko có bản ghi nào có 100%
df['KhiAp (mb)'] = df['KhiAp (mb)'].str[1:-2].astype(int)
df['MatDoMay (%)'] = df['MatDoMay (%)'].str.replace('%','').astype(int)
df['TamNhin (km)'] = df['TamNhin (km)'].str[:-2].astype(float)
df['PM2_Matdo(µg/m³)'] = df['PM2_Matdo(µg/m³)'].str[:-5].astype(float)
df['PM10_Matdo(µg/m³)'] = df['PM10_Matdo(µg/m³)'].str[:-5].astype(float)
df['SO2_Matdo(µg/m³)'] = df['SO2_Matdo(µg/m³)'].str[:-5].astype(float)
df['NO2_Matdo(µg/m³)'] = df['NO2_Matdo(µg/m³)'].str[:-5].astype(float)

#### 2.2: Làm rõ bài toán về các tỉnh thành có nhiều bản ghi khu vực

Ở chương I, ta được biết: tập DL ta thu thập được bao gồm __các bản ghi tổng quan__ và __nhiều bản ghi khu vực của 1 tỉnh thành__, điều này xảy ra là một số tỉnh thành không có trang thời tiết tổng quan.

Đối với các bản ghi loại 2, ta sẽ lấy `giá trị trung bình` các chỉ số đếm được của tất cả các khu vực, đặt thành giá trị tổng quan của tỉnh thành đó.

#### 2.3 Làm rõ cách AccuWeather đưa ra nhận định về trạng thái không khí

AccuWeather có công khai một bảng nhận xét. Căn cứ vào các chỉ số không khí đo được, họ đưa ra nhận xét với từng chỉ số không khí. Vui lòng mở báo cáo để thấy bảng nhận xét đó.
- Các nhận xét trên chính là giá trị các cột `TrangThaiKhongKhi`, `PM2.5_TrangThai`, `PM10_TrangThai`, `SO2_TrangThai`, `NO2_TrangThai`, . 
- Các chỉ số không khí được lưu trong các cột `AQI`, `PM2.5_ChiSo`, `PM10_ChiSo`, `SO2_ChiSo`, `NO2_ChiSo`

Bên dưới là 1 hàm `convert` mô phỏng lại thang đo trên, giúp ta có thể tạo ra lại các cột trạng thái thông qua chỉ số.

In [6]:
def convert(status):
    if 0 <= status <= 19:
        return 'Excellent'
    elif status <= 49:
        return 'Fair'
    elif status <= 99:
        return 'Poor'
    elif status <= 149:
        return 'Unhealthy'
    elif status <= 249:
        return 'Very Unhealthy'
    return 'Dangerous'

#### 2.4 Tạm thời loại bỏ các biến định tính trong tập DL.

Để dễ dàng trong việc tính trung bình các biến liên tục, ta cần tạm thời loại các biến nhận xét trạng thái thời tiết từ chỉ số không khí `TrangThaiKhongKhi`, `PM2.5_TrangThai`, `PM10_TrangThai`, `SO2_TrangThai`, `NO2_TrangThai`. Sau này, ta sẽ tính lại chúng thông qua hàm `convert` trên

In [7]:
df.drop(['KhuVuc','PM10_TrangThai','PM2_TrangThai','SO2_TrangThai','NO2_TrangThai','TrangThaiKhongKhi'], inplace=True, axis=1)

#### 2.5 Tạo ra tập DL mới đáp ứng các hướng giải quyết ở trên

In [8]:
# Lấy ra các cột có giá trị là biến liên tục và đem đi tìm trung bình
mean_columns = df.columns.difference(['ThoiGian', 'TinhThanh'],sort=False)
result = df.groupby(['ThoiGian','TinhThanh'])[mean_columns].mean().reset_index().round(3)

# Đặt lại các trạng thái không khí thông qua chỉ số
phrase_col = ['TrangThaiKhongKhi','PM2_TrangThai', 'PM10_TrangThai', 'SO2_TrangThai', 'NO2_TrangThai']
index_col = ['AQI','PM2_ChiSo','PM10_ChiSo','SO2_ChiSo','NO2_ChiSo']
for phrase, index in zip(phrase_col, index_col):
    result[phrase] = result[index].apply(convert)
    
result.head()

,ThoiGian,TinhThanh,NhietDo (°C),LuongMua (mm),DoAm (%),DoAmTrongNha (%),KhiAp (mb),MatDoMay (%),TamNhin (km),AQI,...,PM10_Matdo(µg/m³),SO2_ChiSo,SO2_Matdo(µg/m³),NO2_ChiSo,NO2_Matdo(µg/m³),TrangThaiKhongKhi,PM2_TrangThai,PM10_TrangThai,SO2_TrangThai,NO2_TrangThai
0,2024-06-07,An Giang,33.143,7.229,59.143,59.143,1008.429,80.429,24.0,29.857,...,16.714,12.571,37.714,6.571,3.429,Fair,Fair,Fair,Excellent,Excellent
1,2024-06-07,Bà Rịa - Vũng Tàu,32.200,27.020,71.000,71.000,1008.000,76.000,24.0,33.600,...,18.200,15.200,46.400,6.200,6.000,Fair,Fair,Fair,Excellent,Excellent
2,2024-06-07,Bình Dương,34.000,17.700,58.857,58.857,1006.571,88.714,24.0,33.143,...,16.286,11.857,34.857,9.714,6.286,Fair,Fair,Fair,Excellent,Excellent
3,2024-06-07,Bình Phước,30.900,5.380,68.500,68.500,1009.300,82.800,23.7,32.700,...,15.500,12.700,37.800,7.100,4.300,Fair,Fair,Fair,Excellent,Excellent
4,2024-06-07,Bình Thuận,31.714,4.390,65.048,65.048,1009.286,82.714,24.0,39.048,...,21.286,17.333,51.571,5.000,3.524,Fair,Fair,Fair,Excellent,Excellent


#### 2.6 Sắp xếp lại các cột của tập DL mới sao cho giống với ban đầu

Hệ quả của việc tách thành các bài toán nhỏ ra khiến vị trí các cột bị xáo trộn 1 chút, ta cần sắp xếp lại

In [9]:
orders = ['ThoiGian', 'TinhThanh','NhietDo (°C)',
          'LuongMua (mm)', 'DoAm (%)', 'DoAmTrongNha (%)', 'KhiAp (mb)', 
          'MatDoMay (%)', 'TamNhin (km)', 'AQI','TrangThaiKhongKhi',
          'PM2_TrangThai', 'PM2_ChiSo', 'PM2_Matdo(µg/m³)',
          'PM10_TrangThai','PM10_ChiSo','PM10_Matdo(µg/m³)',
          'SO2_TrangThai', 'SO2_ChiSo', 'SO2_Matdo(µg/m³)',
          'NO2_TrangThai','NO2_ChiSo','NO2_Matdo(µg/m³)']
result = result.reindex(columns=orders)

#### 2.7 Lưu lại tập dữ liệu đã làm sạch vào một file csv mới

In [10]:
result.to_csv('WeeklyWeather_VN.csv', encoding='utf-8-sig',index=False)

#### 2.8 Lưu lại tập dữ liệu sạch ở phiên bản tiếng Anh

In [11]:
column_mapping = {
    'ThoiGian': 'Time',
    'TinhThanh': 'City',
    'NhietDo (°C)': 'Temperature (°C)',
    'LuongMua (mm)': 'Rainfall (mm)',
    'DoAm (%)': 'Humidity (%)',
    'DoAmTrongNha (%)': 'Indoor Humidity (%)',
    'KhiAp (mb)': 'Pressure (mb)',
    'MatDoMay (%)': 'Cloudiness (%)',
    'TamNhin (km)': 'Visibility (km)',
    'AQI': 'AQI',
    'TrangThaiKhongKhi': 'Air Status',
    'PM2_TrangThai': 'PM2 Status',
    'PM2_ChiSo': 'PM2 Index',
    'PM2_Matdo(µg/m³)': 'PM2 Density (µg/m³)',
    'PM10_TrangThai': 'PM10 Status',
    'PM10_ChiSo': 'PM10 Index',
    'PM10_Matdo(µg/m³)': 'PM10 Density (µg/m³)',
    'SO2_TrangThai': 'SO2 Status',
    'SO2_ChiSo': 'SO2 Index',
    'SO2_Matdo(µg/m³)': 'SO2 Density (µg/m³)',
    'NO2_TrangThai': 'NO2 Status',
    'NO2_ChiSo': 'NO2 Index',
    'NO2_Matdo(µg/m³)': 'NO2 Density (µg/m³)'
}

# Đổi tên các cột
result.rename(columns=column_mapping, inplace=True)
result.to_csv('WeeklyWeather_EN.csv', encoding='utf-8-sig',index=False)